In [ ]:
! pip install -U datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 14.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pla

In [ ]:
!pip install -U transformers datasets evaluate --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 69.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.4 MB/s eta 0:00:00


In [ ]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=5889b68fcd287b1be544ecc2cdc4ad8a9bfd186c80596440b3bfcfd79b729a3a
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [ ]:
import os
import torch
import random
import numpy as np

from datasets import load_dataset
import transformers
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments, AutoModelForTokenClassification, pipeline
import evaluate

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

In [ ]:
os.environ["WANDB_DISABLED"] = "true"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"

In [ ]:
dataset = load_dataset("GEM/schema_guided_dialog")

train_dataset = dataset["train"].select(range(5000))
eval_dataset = dataset["validation"].select(range(1000))

print("Колонки:", train_dataset.column_names)
print("Пример:")
print("Prompt:\n", train_dataset[0]["prompt"])
print("Target:\n", train_dataset[0]["target"])

Колонки: ['gem_id', 'gem_parent_id', 'dialog_acts', 'context', 'dialog_id', 'service', 'turn_id', 'prompt', 'target', 'references', 'linearized_input']
Пример:
Prompt:
 I am feeling hungry so I would like to find a place to eat.
Target:
 Do you have a specific which you want the eating place to be located at?


In [ ]:
model_name = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
# Загрузка модели NER для извлечения слотов
ner_model_name = "dslim/bert-base-NER"
ner_tokenizer = AutoTokenizer.from_pretrained(ner_model_name)
ner_model = AutoModelForTokenClassification.from_pretrained(ner_model_name)
ner_pipeline = pipeline("ner", model=ner_model, tokenizer=ner_tokenizer, aggregation_strategy="simple")

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


In [ ]:
# Используем linearized_input как вход в модель
def preprocess_function(example):
    input_text = example["linearized_input"]

    inputs = tokenizer(
        input_text,
        max_length=128,
        padding="max_length",
        truncation=True,
    )
    targets = tokenizer(
        example["target"],
        max_length=64,
        padding="max_length",
        truncation=True,
    )
    inputs["labels"] = [
        (label if label != tokenizer.pad_token_id else -100)
        for label in targets["input_ids"]
    ]
    return inputs

train_tok = train_dataset.map(preprocess_function, batched=False)
eval_tok = eval_dataset.map(preprocess_function, batched=False)

print("Токенизация прошла успешно, пример токенов:")
print(train_tok[0])

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Токенизация прошла успешно, пример токенов:
{'gem_id': 'schema_guided_dialog-train-0', 'gem_parent_id': 'schema_guided_dialog-train-0', 'dialog_acts': [{'act': 13, 'slot': 'city', 'values': []}], 'context': [], 'dialog_id': '1_00000', 'service': 'Restaurants_1', 'turn_id': 1, 'prompt': 'I am feeling hungry so I would like to find a place to eat.', 'target': 'Do you have a specific which you want the eating place to be located at?', 'references': ['Do you have a specific which you want the eating place to be located at?'], 'linearized_input': 'Prompt: "I am feeling hungry so I would like to find a place to eat.", Response Type: request, Type of Slot: city, Agent: Restaurants_1', 'input_ids': [749, 1167, 17, 10, 96, 196, 183, 1829, 13802, 78, 27, 133, 114, 12, 253, 3, 9, 286, 12, 3, 1544, 535, 6, 16361, 6632, 10, 1690, 6, 6632, 13, 12094, 10, 690, 6, 8628, 10, 6233, 7, 834, 536, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
bleu = evaluate.load("bleu")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)

    pred_str = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(labels, skip_special_tokens=True)

    pred_tokens = [p.split() for p in pred_str]
    label_tokens = [[l.split()] for l in label_str]

    return {"bleu": bleu.compute(predictions=pred_tokens, references=label_tokens)["bleu"]}

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    logging_dir="./logs",
    logging_steps=50,
    eval_strategy="no",
    save_strategy="no",
    report_to="none",
    remove_unused_columns=True,
    fp16=False,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tok,
    eval_dataset=eval_tok,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

torch.cuda.empty_cache()
trainer.train()

model.save_pretrained("./t5-small-finetuned")
tokenizer.save_pretrained("./t5-small-finetuned")

/tmp/ipython-input-20-3280008918.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
50,3.219600
100,2.760500
150,2.536100
200,2.326900
250,2.150800
300,2.113900
350,2.096400
400,2.163300
450,1.961000
500,1.945800


('./t5-small-finetuned/tokenizer_config.json',
 './t5-small-finetuned/special_tokens_map.json',
 './t5-small-finetuned/spiece.model',
 './t5-small-finetuned/added_tokens.json',
 './t5-small-finetuned/tokenizer.json')

In [ ]:
!zip -r t5-small-finetuned.zip t5-small-finetuned
from google.colab import files
files.download("t5-small-finetuned.zip")

  adding: t5-small-finetuned/ (stored 0%)
  adding: t5-small-finetuned/spiece.model (deflated 48%)
  adding: t5-small-finetuned/special_tokens_map.json (deflated 85%)
  adding: t5-small-finetuned/config.json (deflated 63%)
  adding: t5-small-finetuned/tokenizer_config.json (deflated 95%)
  adding: t5-small-finetuned/tokenizer.json (deflated 74%)
  adding: t5-small-finetuned/generation_config.json (deflated 29%)
  adding: t5-small-finetuned/model.safetensors (deflated 11%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>